In [1]:
import numpy as np
import pandas as pd
import math
import re
import warnings as wr
wr.filterwarnings('ignore')

In [3]:
df = (
    pd
    .read_csv("05_flats.csv")
    .drop_duplicates()
    .drop(index=286)
    .apply(lambda x : x.str.strip().str.lower() if x.dtype == 'object' else col)
    .loc[lambda x : x['price'].str.contains('lac|crore', na=False)]
    .assign(
        bhk = lambda x : x['property_name'].str.extract(r"(\d*)")
        ,locality = lambda x : x['property_name'].str.split(" in").apply(lambda x : x[1]).str.strip()
        ,price1 = lambda x : x['price'].str.extract(r"([0-9]*\.*[0-9]*)").astype(float)
        ,price_cr = 
    )
    .drop(columns=['property_name'])
)

df.shape

SyntaxError: expected argument value expression (1267113741.py, line 12)

In [4]:
df.sample(5)

NameError: name 'df' is not defined

In [10]:
df = pd.read_csv("05_flats.csv")
df.drop_duplicates(inplace=True)
df.drop([286], inplace=True)    #removing duplicate header row
#making all column data consistent by converting into lowercase
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip().str.lower()
#creating new cols
df['bhk'] = df['property_name'].str.extract(r"(\d*)")
df['locality'] = df['property_name'].str.split(" in ").apply(lambda x : x[1]).str.strip()
#lac-crore segregation
df = df.iloc[np.where(df['price'].str.contains('lac|crore'))[0]]
df = df[df['price'].str.contains(r"(\d*\.*\d)",na=False)]
df['price_cr'] = df['price'].apply(lambda x : float(re.search(r"(\d*\.*\d*)", str(x)).group())/100 if 'lac' in x else float(re.search(r"(\d*\.*\d*)", str(x)).group()))
df['society_rating'] = df['society'].str.extract(r"(\d{1}\.{1}\d*)")
df['society'] = df['society'].str.split(r"(\d{1}\.{1}\d*)").str[0]
df['area_type'] = df['areaWithType'].str.split(r" area").str[0].str.strip() + r" area"
df['area_sqmt'] = df['areaWithType'].str.extract(r"\({1}(.*?)(?= sq.m.)")[0].str.strip().astype(float)
df['bedRoom'] = df['bedRoom'].str.extract(r"(\d*)")[0].str.strip().astype('int16')
df['bathroom'] = df['bathroom'].str.extract(r"(\d*)")[0].str.strip().astype('int16')
df['balcony'] = df['balcony'].str.extract(r"(\d*)")[0].str.strip().apply(lambda x : 0 if x=='' else int(x))
df['additionalRoom'] = df['additionalRoom'].str.count(r"room").apply(lambda x : 0 if math.isnan(x) else int(x))
df = df[df['floorNum'].str.findall(r"(\d+)").str[1].notna()]
df['building_rise'] = df['floorNum'].str.findall(r"(\d+)").str[1].astype('int16').apply(lambda x : "high rise" if x>4 else "low rise")
df['floor'] = df['floorNum'].str.findall(r"(\d+)").str[0].astype('int16').apply(lambda x : "low" if x<5 else "mid" if x<8 else "high")
df = df[df['agePossession'].notna()]
df = df[(df['agePossession'] != r'under construction') & (df['agePossession'] != r'undefined')]
#removing unnecessary cols
df = df.drop(columns=['property_name','price','area','areaWithType','rating','property_id','address','floorNum'])

df.shape

(2583, 20)

In [11]:
df['agePossession'].unique()

array(['1 to 5 year old', '10+ year old', '0 to 1 year old', 'dec 2023',
       'jun 2024', 'dec 2025', 'by 2023', '5 to 10 year old', 'sep 2025',
       'mar 2024', 'jan 2024', 'within 6 months', 'jan 2027', 'dec 2024',
       'oct 2024', 'nov 2023', 'jul 2026', 'feb 2024', 'oct 2023',
       'jan 2025', 'aug 2023', 'by 2024', 'apr 2024', 'nov 2025',
       'dec 2026', 'within 3 months', 'apr 2026', 'by 2025', 'jul 2027',
       'sep 2023', 'by 2027', 'jan 2026', 'may 2024', 'nov 2024',
       'aug 2024', 'jun 2027', 'mar 2025', 'jul 2024', 'jul 2025',
       'sep 2027', 'jun 2025', 'jan 2028', 'aug 2026', 'mar 2026',
       'aug 2025', 'oct 2025'], dtype=object)

In [19]:
df['agePossession'].apply(
    lambda x :
    1 if 'month' in x else
    2026-int(re.search(r"(\d{4})", str(x)).group()) if re.match(r"(\d{4})", x) else
    re.search(r"(\d+)*year") if re.match(r"year", x) else 0
)

0       0
1       0
2       0
3       0
6       0
       ..
3022    0
3024    0
3025    0
3026    0
3027    0
Name: agePossession, Length: 2583, dtype: int64

In [21]:
re.match('year', "1 to 5 year old")

In [3]:
df

,link,society,bedRoom,bathroom,balcony,additionalRoom,facing,agePossession,nearbyLocations,description,furnishDetails,features,bhk,locality,price_cr,society_rating,area_type,area_sqmt,building_rise,floor
0,https://www.99acres.com/2-bhk-bedroom-apartmen...,maa bhagwati residency,2,2,1,0,west,1 to 5 year old,"['chintapurni mandir', 'state bank atm', 'pear...",so with lift.maa bhagwati residency is one of ...,"['3 fan', '4 light', '1 wardrobe', 'no ac', 'n...","['feng shui / vaastu compliant', 'security / f...",2,krishna colony,0.45,NaN,carpet area,83.61,low rise,low
1,https://www.99acres.com/2-bhk-bedroom-apartmen...,apna enclave,2,2,1,0,west,10+ year old,"['chintapurni mandir', 'sheetla mata mandir', ...","property situated on main road, railway statio...","['3 wardrobe', '4 fan', '1 exhaust fan', '1 ge...","['security / fire alarm', 'maintenance staff',...",2,ashok vihar,0.50,NaN,carpet area,60.39,low rise,low
2,https://www.99acres.com/2-bhk-bedroom-apartmen...,tulsiani easy in homes,2,2,3,0,NaN,0 to 1 year old,"['huda city metro', 'golf course extn road', '...","this property is 15 km away from badshapur, gu...",NaN,"['power back-up', 'feng shui / vaastu complian...",2,sohna,0.40,NaN,carpet area,55.28,high rise,high
3,https://www.99acres.com/2-bhk-bedroom-apartmen...,smart world orchard,2,2,2,1,NaN,dec 2023,"['sector 55-56 metro station', 'bestech centra...",near to metro station of sector 56 and opposit...,NaN,"['security / fire alarm', 'private garden / te...",2,sector 61 gurgaon,1.47,NaN,carpet area,111.48,low rise,low
4,https://www.99acres.com/2-bhk-bedroom-apartmen...,parkwood westend,2,2,3,1,NaN,under construction,"['yadav clinic', 'bangali clinic', 'dr. j. s. ...",we are the proud owners of this 2 bhk alongwit...,[],NaN,2,sector 92 gurgaon,0.70,NaN,super built up area,124.95,high rise,mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023,https://www.99acres.com/3-bhk-bedroom-apartmen...,ansal heights 86,3,3,3,1,north-east,under construction,"['iris broadway mall', 'delhi jaipur expresswa...",residential apartment for sell.located in sect...,[],NaN,3,sector 86 gurgaon,1.05,NaN,super built up area,176.05,high rise,high
3024,https://www.99acres.com/5-bhk-bedroom-apartmen...,parsvnath green ville,5,5,3,1,NaN,10+ year old,"['sri radhe krishna temple', 'icici bank atm',...",5 bhk duplex penthouse in low rise building.av...,NaN,"['security / fire alarm', 'private garden / te...",5,sector 48 gurgaon,3.30,3.9,super built up area,362.79,high rise,low
3025,https://www.99acres.com/3-bhk-bedroom-apartmen...,raheja vedaanta,3,3,3,0,NaN,1 to 5 year old,NaN,3 bedroom flat with full woodwork. ready to mo...,[],"['security / fire alarm', 'feng shui / vaastu ...",3,sector 108 gurgaon,0.95,3.6,super built up area,169.27,high rise,low
3026,https://www.99acres.com/3-bhk-bedroom-apartmen...,ambience lagoon,3,4,3,3,north-east,10+ year old,"['micromax moulsari avenue metro station', 'in...",luxury condominium complex located on delhi gu...,"['1 water purifier', '10 fan', '1 fridge', '1 ...","['security / fire alarm', 'private garden / te...",3,dlf phase 3,5.80,3.9,built up area,343.74,high rise,high
